In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import wandb
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("/home/ubuntu/ayusht/skin/data/extra/extra_metadata.csv")

/tmp/ipykernel_2174929/94633858.py:1: DtypeWarning: Columns (8,13,16,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/ubuntu/ayusht/skin/data/extra/extra_metadata.csv")


In [4]:
df.head()

,isic_id,attribution,copyright_license,acquisition_day,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,concomitant_biopsy,dermoscopic_type,...,mel_thick_mm,mel_type,mel_ulcer,melanocytic,nevus_type,patient_id,personal_hx_mm,pixels_x,pixels_y,sex
0,ISIC_7559201,Memorial Sloan Kettering Cancer Center,CC-BY,2497.0,55.0,anterior torso,benign,6.6,NaN,contact non-polarized,...,NaN,NaN,NaN,True,NaN,IP_1238256,True,3264,2448,female
1,ISIC_0485014,Memorial Sloan Kettering Cancer Center,CC-BY,1.0,45.0,lower extremity,benign,NaN,NaN,contact non-polarized,...,NaN,NaN,NaN,True,NaN,IP_3227071,NaN,6000,4000,female
2,ISIC_5257439,Memorial Sloan Kettering Cancer Center,CC-BY,2360.0,40.0,lateral torso,benign,4.2,NaN,NaN,...,NaN,NaN,NaN,True,NaN,IP_7407753,True,3264,2448,female
3,ISIC_2989732,Memorial Sloan Kettering Cancer Center,CC-BY,78.0,80.0,anterior torso,benign,NaN,NaN,non-contact polarized,...,NaN,NaN,NaN,True,NaN,IP_2597637,NaN,6000,4000,male
4,ISIC_5638210,Memorial Sloan Kettering Cancer Center,CC-BY,78.0,80.0,anterior torso,benign,NaN,NaN,contact non-polarized,...,NaN,NaN,NaN,True,NaN,IP_2597637,NaN,6000,4000,male


In [5]:
len(df)

81722

In [6]:
df.benign_malignant.value_counts()

benign_malignant
benign                     64047
malignant                   9239
indeterminate                150
indeterminate/malignant       85
indeterminate/benign          67
Name: count, dtype: int64

In [61]:
malignant_df = df.loc[df.benign_malignant == "malignant"]

In [62]:
malignant_df.columns

Index(['isic_id', 'attribution', 'copyright_license', 'acquisition_day',
       'age_approx', 'anatom_site_general', 'benign_malignant',
       'clin_size_long_diam_mm', 'concomitant_biopsy', 'dermoscopic_type',
       'diagnosis', 'diagnosis_confirm_type', 'family_hx_mm',
       'fitzpatrick_skin_type', 'image_type', 'lesion_id', 'mel_class',
       'mel_mitotic_index', 'mel_thick_mm', 'mel_type', 'mel_ulcer',
       'melanocytic', 'nevus_type', 'patient_id', 'personal_hx_mm', 'pixels_x',
       'pixels_y', 'sex'],
      dtype='object')

In [22]:
malignant_imgs = glob("../data/extra/malignant_images/*.jpg")
len(malignant_imgs)

9239

In [23]:
f = glob("../data/extra/malignant_images/*")
len(f)

9242

In [26]:
for img in f:
    if img.endswith(".jpg"):
        continue
    else:
        print(img)

../data/extra/malignant_images/metadata.csv
../data/extra/malignant_images/attribution.txt
../data/extra/malignant_images/licenses


In [27]:
_malignant_df = pd.read_csv("../data/extra/malignant_images/metadata.csv")

In [63]:
malignant_df.head()

,isic_id,attribution,copyright_license,acquisition_day,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,concomitant_biopsy,dermoscopic_type,...,mel_thick_mm,mel_type,mel_ulcer,melanocytic,nevus_type,patient_id,personal_hx_mm,pixels_x,pixels_y,sex
43,ISIC_1633321,Memorial Sloan Kettering Cancer Center,CC-BY,1121.0,80.0,anterior torso,malignant,10.1,NaN,contact non-polarized,...,NaN,NaN,NaN,True,NaN,IP_2597637,True,3264,2448,male
54,ISIC_2222766,Memorial Sloan Kettering Cancer Center,CC-BY,1233.0,80.0,lower extremity,malignant,10.4,NaN,NaN,...,NaN,NaN,NaN,True,NaN,IP_2597637,True,3264,2448,male
55,ISIC_6720909,Memorial Sloan Kettering Cancer Center,CC-BY,1233.0,80.0,lower extremity,malignant,10.4,NaN,contact non-polarized,...,NaN,NaN,NaN,True,NaN,IP_2597637,True,3264,2448,male
56,ISIC_6364782,Memorial Sloan Kettering Cancer Center,CC-BY,1233.0,80.0,lower extremity,malignant,10.4,NaN,contact polarized,...,NaN,NaN,NaN,True,NaN,IP_2597637,True,3264,2448,male
67,ISIC_2792717,Memorial Sloan Kettering Cancer Center,CC-BY,1121.0,80.0,anterior torso,malignant,10.1,NaN,contact polarized,...,NaN,NaN,NaN,True,NaN,IP_2597637,True,3264,2448,male


In [64]:
malignant_df = malignant_df.reset_index(drop=True)
malignant_df["target"] = 1

In [65]:
from sklearn.model_selection import KFold

gkf = KFold(n_splits=5) # , shuffle=True, random_state=42

malignant_df["fold"] = -1
for idx, (train_idx, val_idx) in enumerate(gkf.split(malignant_df)):
    malignant_df.loc[val_idx, "fold"] = idx

In [67]:
malignant_df.to_csv("../data/extra_malignant.csv", index=False)

In [76]:
# Data
def add_path(row):
    return f"../data/train-image/image/{row.isic_id}.jpg"


def add_extra_path(row):
    return f"../data/extra/malignant_images/{row.isic_id}.jpg"

train_metadata_df = pd.read_csv("../data/stratified_5_fold_train_metadata.csv")
extra_malignant_df = pd.read_csv("../data/extra_malignant.csv")
train_metadata_df["path"] = train_metadata_df.apply(lambda row: add_path(row), axis=1)
extra_malignant_df["path"] = extra_malignant_df.apply(lambda row: add_extra_path(row), axis=1)
print(f"Train: {len(train_metadata_df)} | Extra train: {len(extra_malignant_df)}")

train_metadata_df = train_metadata_df[["path", "target", "fold", "isic_id", "patient_id"]]
extra_malignant_df = extra_malignant_df[["path", "target", "fold", "isic_id", "patient_id"]]

/tmp/ipykernel_2174929/1601854888.py:9: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata_df = pd.read_csv("../data/stratified_5_fold_train_metadata.csv")


Train: 401059 | Extra train: 9239


In [77]:
train_df_1 = train_metadata_df.loc[
    train_metadata_df.fold == 0
]  # using a subset for training
train_df_2 = train_metadata_df.loc[
    train_metadata_df.fold == 2
]  # using a subset for training
train_df_3 = train_metadata_df.loc[
    train_metadata_df.fold == 3
]  # using a subset for training
train_df = pd.concat([train_df_1, train_df_2, train_df_3])
valid_df = train_metadata_df.loc[
    train_metadata_df.fold == 1
]  # using another fold for validation

In [74]:
t_ids = train_df.isic_id.values
v_ids = valid_df.isic_id.values

In [75]:
for v_id in v_ids:
    if v_id in t_ids:
        print(v_id)

In [78]:
t_ids = train_df.patient_id.values
v_ids = valid_df.patient_id.values

In [79]:
for v_id in v_ids:
    if v_id in t_ids:
        print(v_id)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def loss_function(x, y, delta):
    diff = x - y
    if abs(diff) < delta:
        return 0.5 * diff**2
    else:
        return delta * (abs(diff) - 0.5 * delta)

def plot_loss(delta=1.0, y=0):
    x = np.linspace(-5, 5, 1000)
    l = [loss_function(xi, y, delta) for xi in x]
    
    plt.figure(figsize=(10, 6))
    plt.plot(x, l)
    plt.title(f'Loss Function (delta={delta}, y={y})')
    plt.xlabel('x')
    plt.ylabel('loss')
    plt.grid(True)
    plt.show()

interact(plot_loss, 
         delta=FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0),
         y=FloatSlider(min=-5.0, max=5.0, step=0.1, value=0.0))

interactive(children=(FloatSlider(value=1.0, description='delta', max=5.0, min=0.1), FloatSlider(value=0.0, de…

<function __main__.plot_loss(delta=1.0, y=0)>